# *Dynamic Routing*
<img src="https://www.kvc.org/wp-content/uploads/2015/08/brain-600px-600x372.jpg" width="320" />

## Run the update/reset shortcut on the desktop before each experiment
***
***
# **Without mouse on stage**

In [3]:
import pathlib

import IPython
import ipywidgets as widgets

import np_logging
import np_session
import np_services
import np_workflows
from np_workflows import npxc
import np_workflows.experiments.dynamic_routing as DR

np_logging.getLogger()

np_workflows.elapsed_time_widget()

Not connected to a rig: `OpenEphys.host` needs to be set manually.


ValueError: Rig index not specified and not running on a rig.

***
## Quiet mode
on  
*default*
- exception details are hidden
- regular messages displayed (log level = INFO)

off
- full exception details with traceback
- extra messages displayed (log level = DEBUG)

In [ ]:
np_workflows.quiet_mode_widget()

***
## Launch apps via RSC
[optional]

In [ ]:
import contextlib
with contextlib.suppress(Exception):
    np_services.start_rsc_apps()

***
## Select mouse and user

In [ ]:
user, mouse = np_workflows.user_and_mouse_widget()

***
## Select workflow
Re-run this cell if mouse ID is changed

In [ ]:
selection = DR.workflow_select_widget(mouse)

***
## Check MTrain for mouse
[optional]

In [ ]:
np_workflows.mtrain_widget(mouse)

***
## Generate new session
Re-run this cell if workflow is changed above.

Check mouse ID and session are correct: this cell will lock them in!

In [ ]:
experiment: DR.Hab | DR.Ephys = DR.new_experiment(mouse, user, selection.workflow)

session: np_session.Session = experiment.session

hab: bool = isinstance(experiment, DR.Hab)

***
## Set task - *critical!*

double-check this is correct and the cell has been run


In [ ]:
experiment.task_name = 'stage 5 ori AMN moving timeouts repeats'

***
## Pretest
Checks before running

In [ ]:
np_workflows.check_hardware_widget()

In [ ]:
if not hab:
    np_workflows.check_openephys_widget()

***
## Setup, test, reset all components
*This cell must not be skipped!*

In [ ]:
assert experiment.task_name, "task name has not been set - don't continue until running earlier cell"
experiment.initialize_and_test_services()

***
## Run sound test

In [ ]:
experiment.run_sound_test()

***
## Dip probes

In [ ]:
if not hab:
    np_workflows.dye_widget(session.npexp_path)
    DR.photodoc_widget(session, 'pre_experiment_surface_image')

***
***
# **With mouse on stage**
## Before lowering cartridge

In [ ]:
experiment.log('Mouse on stage')
np_workflows.check_mouse_widget()

***
## When cartridge is lowered

In [ ]:
DR.photodoc_widget(session, 'brain_surface_image')

***
## Probe insertion

In [ ]:
import np_probe_targets

np_probe_targets.DRWeeklyTargets(1)

for targets in (_ for _ in DRWeeklyTargets.children if isinstance(_, np_probe_targets.ProbeTargetInsertionRecordWidget)):
    targets.save_dir = experiment.npexp_path

In [ ]:
if not hab:
    np_workflows.isi_widget(mouse.lims)

***
## Photodoc before advancing probes

In [ ]:
if not hab:
    DR.photodoc_widget(session, 'pre_insertion_surface_image')

***
## Settle timer with dark screen (screen should be closed)

In [ ]:
if not hab:
    experiment.set_dark_desktop_on_stim()
    experiment.log('settle timer started')
    np_workflows.print_countdown_timer(minutes=.1 if experiment.workflow.value == 'pretest' else 20)
    experiment.log('settle timer finished')

***
## Use OptoGUI on Vidmon to adjust laser and save voltages
This next cell will confirm that the voltages file can be found - *optotagging
won't be able to run without it!*

In [ ]:
if not hab:
    assert pathlib.Path(experiment.optotagging_params['optoTaggingLocs']).exists(), 'voltages file not found - do not continue if optotagging is part of this experiment!'

***
## Photodoc after probes settled, before experiment

In [ ]:
if not hab:
    DR.photodoc_widget(session, 'post_insertion_surface_image')

In [ ]:
np_workflows.pre_stim_check_widget()

***
## Start devices recording

In [ ]:
experiment.start_recording()

***
## Without lick spout
### mapping | optotagging | spontaneous

In [ ]:
np_services.MouseDirector.get_proxy().retract_lick_spout()
experiment.run_mapping()
if not hab:
    experiment.run_optotagging()
    experiment.run_spontaneous()

*** 
## With lick spout
- set mouse offset
- flush line, check water delivery

In [ ]:
np_services.MouseDirector.get_proxy().extend_lick_spout()

### spontaneous + rewards | task | spontaneous + rewards
Check lickspout is extended before starting!

In [ ]:
experiment.run_spontaneous_rewards()
experiment.run_task()
if not hab:
    experiment.run_spontaneous_rewards()

***
## Without lickspout
### optotagging

In [ ]:
if not hab:
    np_services.MouseDirector.get_proxy().retract_lick_spout()
    experiment.run_optotagging()

***
## Stop recording

In [ ]:
experiment.stop_recording_after_stim_finished()
experiment.reset_desktop_on_stim()

***
## Before removing probes

In [ ]:
if not hab:
    DR.photodoc_widget(session, 'post_stimulus_surface_image')

***
## After fully retracting probes

In [ ]:
if not hab:
    DR.photodoc_widget(session, 'post_experiment_surface_image')

***
## After raising cartridge

In [ ]:
np_workflows.finishing_checks_widget()

## Record mouse off stage time

In [ ]:
experiment.log('Mouse off stage')

***

## Finalize

In [ ]:
experiment.finalize_services(*experiment.recorders, *experiment.stims)
experiment.validate_services(*experiment.recorders, *experiment.stims)

## Copy data

In [ ]:
experiment.copy_files()